In [111]:
from keras import layers
from keras.models import Model, Sequential
from keras.utils import plot_model
from keras import backend as K
#from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython import display

In [112]:
def res_block(y, nb_channels, _strides = (1,1), _project_shortcut=False):
    shortcut = y

    y = layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
    #y = layers.BatchNormalization()(y)
    y = layers.ReLU()(y)

    y = layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
    #y = layers.BatchNormalization()()

    if _project_shortcut or _strides != (1, 1):
        shortcut = layers.Conv2D(nb_channels, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    y = layers.add([shortcut, y])
    #y = layers.LeakyReLU()(y)

    return y

In [113]:
def res_net(x, nb_channels, _strides=(1, 1)):
    x = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(x)
    shortcut = x
    for _ in range(16):
        x = res_block(x, 64)

    x = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(x)
    x = layers.add([shortcut, x])
    
    return x

In [114]:
def conv_net(x, nb_channels, _strides=(1, 1)):
    x = layers.Conv2D(32, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(x)
    #x = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(x)
    
    return x

In [115]:
def post_net(y, nb_channels, _strides=(1, 1)):
    #y = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(y)
    #y = layers.Conv2D(32, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(y)
    y = layers.Conv2D(3, kernel_size=(3, 3), strides=_strides, padding='same', activation='linear')(y)
    
    return y

In [116]:
import cv2
import numpy as np

def load_imgs(path, number, train_type):
    result=np.empty((number, 64, 64, 3), dtype="float64")
    for i in range(number):
        I = cv2.imread(path + "{:04}_{}.jpeg".format(i+1, train_type))
        result[i, :, :, :] = I
    return result/result.max()

In [117]:
#inport training data
dataNum = 1000
x1_train = load_imgs("./blurImg/", dataNum, 1)
x2_train = load_imgs("./blurImg/", dataNum, 2)
y_train = load_imgs("./blurImg/", dataNum, 0)

def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
        
def loss_wrapper(in_tensor1, in_tensor2):
    def gaussian_blur(in_tensor):
        # use large kernel to blur pred and in_tensor//
        return
        
    def custom_loss(y_true, y_pred):
        # or better implementation like fourier transformation
        return K.binary_crossentropy(y_true, y_pred) + K.reduce_mean(K.square(gaussian_blur(y_pred)-gaussian_blur(in_tensor1)))
    return custom_loss

In [118]:
img_a = layers.Input(shape=(64, 64, 3))
img_b = layers.Input(shape=(64, 64, 3))
#feature_a = conv_net(img_a, 3)
#feature_b = conv_net(img_b, 3)
feature_a = res_net(img_a, 3)
feature_b = res_net(img_b, 3)
merge = layers.concatenate([feature_a, feature_b])
aif = post_net(merge, 128)
gen = Model(inputs = [img_a, img_b], output = [aif])
#gen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#gen.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
gen.summary()
#plot_model(gen, to_file='generator.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_829 (Conv2D)             (None, 64, 64, 64)   1792        input_25[0][0]                   
__________________________________________________________________________________________________
conv2d_863 (Conv2D)             (None, 64, 64, 64)   1792        input_26[0][0]                   
__________________________________________________________________________________________________
conv2d_830

/home/qian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  if __name__ == '__main__':


In [119]:
cv2.imwrite("a.jpg", 255.0*(gen_img[4]-gen_img[4].min())/(gen_img[4].max()-gen_img[4].min()))

True

In [120]:
image_fake = gen([img_a, img_b])
dis = Sequential()
dis.add(layers.Conv2D(64, kernel_size=(3, 3), padding='same'))
dis.add(layers.LeakyReLU())
dis.add(layers.Conv2D(128, kernel_size=(3, 3), padding='same'))
dis.add(layers.LeakyReLU())
dis.add(layers.Conv2D(256, kernel_size=(3, 3), padding='same'))
dis.add(layers.LeakyReLU())
dis.add(layers.Conv2D(1, kernel_size=(3, 3), padding='same'))

dis.add(layers.Flatten())
dis.add(layers.Dense(512, activation='tanh'))
dis.add(layers.Dense(1))
dis.add(layers.Activation('sigmoid'))
pred_prob = dis(image_fake)
dis.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
dis.summary()
plot_model(dis, to_file='discriminator.png')
make_trainable(dis, False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_898 (Conv2D)          (None, 64, 64, 64)        1792      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_899 (Conv2D)          (None, 64, 64, 128)       73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_900 (Conv2D)          (None, 64, 64, 256)       295168    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_901 (Conv2D)          (None, 64, 64, 1)         2305      
__________

In [121]:
am = Model(inputs = [img_a, img_b], output = [pred_prob])
am.summary()
am.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
plot_model(am, to_file='adversary.png')

/home/qian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
model_13 (Model)                (None, 64, 64, 3)    2444291     input_25[0][0]                   
                                                                 input_26[0][0]                   
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1)            2471298     model_13[1][0]                   
Total para

In [122]:
#pre-train discriminate network


In [123]:
def plot_loss(losses):
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.figure(figsize=(10,8))
        plt.plot(losses["d"], label='discriminitive loss')
        plt.plot(losses["g"], label='generative loss')
        plt.legend()
        plt.show()

In [125]:
# Train discriminator on generated images
losses = {"d":[], "g":[]}
Batch_size = 64
nb_epoch = 100
for epoch in range(nb_epoch):
    rand_idx = np.random.randint(0, x1_train.shape[0], size = Batch_size)
    img_batch1 = x1_train[rand_idx, :, :, :]
    img_batch2 = x2_train[rand_idx, :, :, :]
    y_batch = y_train[np.random.randint(0, y_train.shape[0], size = Batch_size), :, :, :]
    #gen.fit([x1_train, x2_train], y_train)
    gen_img = gen.predict([img_batch1, img_batch2])
    X = np.concatenate((y_batch, gen_img))
    y = np.zeros([2*Batch_size,])
    y[0:Batch_size] = 1
    y[Batch_size:] = 0
    make_trainable(dis,True)
    d_loss = dis.train_on_batch(X, y)
    losses["d"].append(d_loss)
    
    y2 = np.ones([Batch_size, ])
    # train Generator-Discriminator stack on input noise to non-generated output class
    make_trainable(dis,False)
    g_loss = am.train_on_batch([img_batch1, img_batch1], y2) #same batch or ???
    losses["g"].append(g_loss)
    if epoch % 25 == 25 - 1:
        plot_loss(losses)

ResourceExhaustedError: OOM when allocating tensor with shape[64,64,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: model_13/conv2d_883/convolution = Conv2D[T=DT_FLOAT, _class=["loc:@train...kpropInput"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](model_13/re_lu_410/Relu, conv2d_883/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_12/mul/_14309 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7380_loss_12/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated t

In [94]:
Batch_size = 64
nb_epoch = 100
for epoch in range(nb_epoch):
    rand_idx = np.random.randint(0, x1_train.shape[0], size = Batch_size)
    img_batch1 = x1_train[rand_idx, :, :, :]
    img_batch2 = x2_train[rand_idx, :, :, :]
    y_batch = y_train[rand_idx, :, :, :]
    gen.fit([img_batch1, img_batch2], y_batch)

Epoch 1/1
64/64 [==============================] - 8s 119ms/step - loss: 1201.6720 - acc: 0.3019
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.3614 - acc: 0.3938
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 24.9446 - acc: 0.2841
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 134.7787 - acc: 0.2597
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 6.3969 - acc: 0.5315
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 2.7259 - acc: 0.4355
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 6.2487 - acc: 0.3875
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 1.0824 - acc: 0.3719
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.6969 - acc: 0.1989
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.3318 - acc: 0.3700
Epoch 1/1
64/64 [==============================] - 2s 33ms/step

64/64 [==============================] - 2s 34ms/step - loss: 0.0081 - acc: 0.6707
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0076 - acc: 0.6237
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0096 - acc: 0.6728
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0099 - acc: 0.6385
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0088 - acc: 0.6533
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0077 - acc: 0.6380
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0086 - acc: 0.5878
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0082 - acc: 0.6333
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0077 - acc: 0.6834
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0082 - acc: 0.6822
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 0.0083 -

In [110]:
#gen_img[0]
#gen_img.min()
gen_img = gen.predict([x1_train, x2_train])

KeyboardInterrupt: 